<a href="https://colab.research.google.com/github/GerardoToso/Competencia-FSadosky-e-YPF-/blob/main/02_Baseline_toda_la_poblacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">  
           <img 
               src="" 
               style="width:10px"
           >
           </a>    
       </td>
   </tr>
</table>
<div style="float:right;"><h2>02. Baseline toda la población</h2></div>

 <a id="top"></a> <br>
## Índice
1. [Entradas y Entregable](#1)<br>
    1.1. [Entradas del modelo](#11)<br>
    1.2. [Entregable](#12)<br>
2. [Carga de paquetes y datos](#2)<br>
    2.1. [Librerías y clases](#21)<br>
    2.2. [Lectura de datos](#22)<br>
    2.3. [Eliminación de valores nulos](#23)<br>
    2.4. [Separación en train y test](#24)<br>
3. [Modelo](#3)<br>
    3.1. [Confección pipeline](#31)<br>
    3.2. [Optimización de hiperparámetros](#32)<br>
    3.3. [Visualizaciones de hiperparámetros](#33)<br>
    3.4. [Entrenamiento del modelo con mejores hiperparámetros](#34)<br>
4. [Evaluación del modelo](#4)<br>
    4.1. [Cálculo de métricas](#41)<br>
    4.2. [Matriz de confusión](#42)<br>
    4.3. [ROC Curve](#43)<br>
    4.4. [Precision-Recall Curve](#44)<br>
    4.5. [Feature importance](#45)<br>
5. [Exportación con shelve](#5)<br>
    5.1. [Exportación con shelve](#51)<br>

***
<a id="1"></a> <br>
## 1. Entradas y Entregable

<a id="11"></a>
### 1.1. Entradas

|Tipo|Nombre|Descripción|Ubicación Lab|
|:-------|----------------|---------------------------------------------------|-------------------------------------|
|**csv**| `Dataset participantes.csv` | datos relacionados con los pozos al producirse una interferencia por la estimulación de un pozo hijo |`/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/`|

<a id="12"></a>
### 1.2. Entregable

**Archivo csv con las estimaciones de delta_WHP**

***
<a id="2"></a> <br>
## 2. Carga de paquetes y datos

<a id="21"></a>
### 2.1. Librerías y clases

In [ ]:
! pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib. pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler,  FunctionTransformer, LabelEncoder, binarize
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
import lightgbm as lgb

import optuna

import chardet
%matplotlib inline

from IPython.display import Image
import pydotplus

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Esta clase permite dropear columnas de un DataFrame dentro de un pipeline

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self  

<a id="22"></a> <br>
### 2.2. Lectura de datos

In [ ]:
# Detección automática de la codificación de los caracteres
with open("/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/Dataset participantes.csv", 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large

# Lectura del csv
df = pd.read_csv("/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/Dataset participantes.csv", encoding = enc['encoding'],sep='\t', index_col=0)

df.sample(5)

<a id="23"></a> <br>
### 2.3. Eliminación de valores nulos
**Dado que el modelo a desarrollar en el presente notebook se tomará como baseline, no se incluyen en la solución los valores nulos**

In [ ]:
print("Cantidad de instancias previo a la eliminación de valores nulos: ", df.shape[0])
df.dropna(inplace= True)
print("Cantidad de instancias posterior a la eliminación de valores nulos: ", df.shape[0])

<a id="24"></a> <br>
### 2.4. Separación en train y test

In [ ]:
# Selección de variable target
y = df.delta_WHP
# Selección de features
X = df.drop(["delta_WHP"], axis = 1)

# Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 161)

# Verificación de distribución de variable targuet en train y test
print("Instancias de train \n", X_train.shape[0])
print("Instancias de test \n", X_test.shape[0])

***
<a id="#3"></a> <br>
## 3. Modelo

In [ ]:
#install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#check python version
!python --version
#3.9.6

<a id="31"></a> <br>
### 3.1. Confección pipeline

In [ ]:
# Instanciado de estimador
lgbm = lgb.LGBMRegressor(random_state = 45)

# Instanciado de pipeline
pipe = Pipeline[ ("column_dropper", ColumnDropper(["ID_EVENTO"])),
                  
                              ("column_transformer",ColumnTransformer([
                                         ("one_hot_encoder", OneHotEncoder(),   
                    ["CAMPO", "FLUIDO", "PAD_HIJO", "HIJO", "ETAPA_HIJO", "PADRE", "ESTADO"]),
                                        ("standard_scaler", StandardScaler(), 
                    ["D3D", "D2D", "DZ", "AZ", "WHP_i", "#_BARRERAS"])])),
                                 
                                 ("lgbm", lgbm)]

<a id="32"></a> <br>
### 3.2. Optimización de hiperparámetros

In [ ]:
# Función objetivo de optuna

def objective_pipe(trial):

  # Tuneo de hiperparámetros
  max_depth_lgbm = trial.suggest_int("max_depth_lgbm", 5, 50) # lgbm
  learning_rate_lgbm = trial.suggest_float("learning_rate_lgbm", 0.001, 0.08) # lgbm
  n_estimators_lgbm = trial.suggest_int("n_estimators_lgbm", 5, 100) # lgbm
    
    
    
  # Instanciado del estimador
  light_gbm = lgb.LGBMClassifier(random_state = 45, max_depth = max_depth_lgbm, learning_rate = learning_rate_lgbm,
                                  n_estimators = n_estimators_lgbm)

  # Instanciado de pipeline
  pipe = Pipeline[ ("column_dropper", ColumnDropper(["ID_EVENTO"])),
                  
                              ("column_transformer",ColumnTransformer([
                                         ("one_hot_encoder", OneHotEncoder(),   
                    ["CAMPO", "FLUIDO", "PAD_HIJO", "HIJO", "ETAPA_HIJO", "PADRE", "ESTADO"]),
                                        ("standard_scaler", StandardScaler(), 
                    ["D3D", "D2D", "DZ", "AZ", "WHP_i", "#_BARRERAS"])])),
                                 
                                 ("lgbm", lgbm)]
  # Cross validation
  kfold=StratifiedKFold(n_splits=3, random_state=1,shuffle=True)    
    
  # Return
  return cross_val_score(pipe, X_train, y_train, cv = kfold, scoring = mean_squared_error ).mean()  

In [ ]:
%%time
models=[]
scores=[]
pipe_study = optuna.create_study(direction="minimize")
pipe_study.optimize(objective_pipe, n_trials=10)
model="light_gbm"
score=pipe_study.best_trial.value
models.append(model)
scores.append(score)

In [ ]:
print('Number of finished trials:', len(pipe_study.trials))
print('Best trial: score {}, params {}'.format(pipe_study.best_trial.value, pipe_study.best_trial.params))

<a id="33"></a> <br>
### 3.3. Visualizaciones de hiperparámetros

In [ ]:
optuna.visualization.plot_optimization_history(pipe_study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(pipe_study)

In [ ]:
optuna.visualization.plot_slice(pipe_study)

In [ ]:
optuna.visualization.plot_param_importances(pipe_study)

In [ ]:
optuna.visualization.plot_edf(pipe_study)

In [ ]:
optuna.visualization.plot_contour(pipe_study)


<a id="34"></a> <br>
### 3.4. Entrenamiento del modelo con mejores hiperparámetros

In [ ]:
pipe_study.best_trial.params

In [ ]:
# Seteado de mejores hiperparametros en el pipeline
best_params = { 
               "lgbm__max_depth" : pipe_study.best_trial.params['max_depth_lgbm'],
               "lgbm__learning_rate" :  pipe_study.best_trial.params['learning_rate_lgbm'],
               "lgbm__n_estimators" : pipe_study.best_trial.params["n_estimators_lgbm"]
                }

pipe.set_params(**best_params)

In [ ]:
model = pipe.fit(X_train, y_train)

In [ ]:
model.get_params()